## 1. Read the data from the CSV file

#### 1.1 Running imports:

In [1]:
import os
from dotenv import load_dotenv
import psycopg2
from sqlalchemy import create_engine, text
import pandas as pd

#### 1.2 Load database credentials from environment variables:

In [2]:
load_dotenv()

db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = int(os.getenv("DB_PORT", 5432))
db_name = os.getenv("DB_NAME")

if db_user is None:
    raise ValueError("DB_USER is not set")
if db_password is None:
    raise ValueError("DB_PASSWORD is not set")
if db_host is None:
    raise ValueError("DB_HOST is not set")
if db_port is None:
    raise ValueError("DB_PORT is not set")
if db_name is None:
    raise ValueError("DB_NAME is not set")

conn = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)
conn.autocommit = True   

#### 1.3 Read the CSV file:

In [3]:
df_candidates = pd.read_csv("candidates.csv", delimiter=";", skiprows=1)

# Rename columns after loading the data
df_candidates.columns = ["first_name", "last_name", "email", "application_date", "country", "yoe", "seniority", "technology", "code_challenge_score", "technical_interview_score"]

# Just print the first 20 rows to test
df_candidates.head(20)

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score
0,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
1,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
2,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
3,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7
4,Alec,Abbott,juanita_hansen@gmail.com,2019-08-17,Zimbabwe,8,Junior,Adobe Experience Manager,2,9
5,Allison,Jacobs,alba_rolfson27@yahoo.com,2018-05-18,Wallis and Futuna,19,Trainee,Sales,2,9
6,Nya,Skiles,madisen.zulauf@gmail.com,2021-12-09,Myanmar,1,Lead,Mulesoft,2,5
7,Mose,Lakin,dale_murazik@hotmail.com,2018-03-13,Italy,18,Lead,Social Media Community Management,7,10
8,Terrance,Zieme,dustin31@hotmail.com,2022-04-08,Timor-Leste,25,Lead,DevOps,2,0
9,Aiyana,Goodwin,vallie.damore@yahoo.com,2019-09-22,Armenia,24,Intern,Development - CMS Backend,4,9


# 2. Load the data into the database (staging area):

#### 2.1 Create the database engine:

In [4]:
db_engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

#### 2.2 Store the CSV file data into a PostgreSQL database:

In [5]:
with db_engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS candidates (
            first_name CHARACTER VARYING,
            last_name CHARACTER VARYING,
            email CHARACTER VARYING,
            application_date DATE,
            country CHARACTER VARYING,
            yoe INTEGER,
            seniority CHARACTER VARYING,
            technology CHARACTER VARYING,
            code_challenge_score INTEGER,
            technical_interview_score INTEGER
        );
    """))
    conn.commit()
    print("Table 'candidates' created successfully.")

# Store DataFrame into PostgreSQL table named 'candidates'
df_candidates.to_sql("candidates", db_engine, if_exists="append", index=False)

print("The CSV file data was inserted successfully!")

Table 'candidates' created successfully.
The CSV file data was inserted successfully!


# 3. Read the table that was created with Python:

In [6]:
# Just print the first 20 rows to test
df_verification = pd.read_sql("SELECT * FROM candidates LIMIT 20;", db_engine)
df_verification

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score
0,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
1,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
2,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
3,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7
4,Alec,Abbott,juanita_hansen@gmail.com,2019-08-17,Zimbabwe,8,Junior,Adobe Experience Manager,2,9
5,Allison,Jacobs,alba_rolfson27@yahoo.com,2018-05-18,Wallis and Futuna,19,Trainee,Sales,2,9
6,Nya,Skiles,madisen.zulauf@gmail.com,2021-12-09,Myanmar,1,Lead,Mulesoft,2,5
7,Mose,Lakin,dale_murazik@hotmail.com,2018-03-13,Italy,18,Lead,Social Media Community Management,7,10
8,Terrance,Zieme,dustin31@hotmail.com,2022-04-08,Timor-Leste,25,Lead,DevOps,2,0
9,Aiyana,Goodwin,vallie.damore@yahoo.com,2019-09-22,Armenia,24,Intern,Development - CMS Backend,4,9


# 4. Do some transformations:

#### 4.1 Create a copy of the candidates data frame and add the hired column:

In [7]:
df_candidates_transformed = df_candidates.copy()
df_candidates_transformed["hired"] = df_candidates_transformed.apply(
    lambda x: "Yes" if x["code_challenge_score"] >= 7 and x["technical_interview_score"] >= 7 else "No",
    axis=1
) 

# Just print the first 20 rows to test
df_candidates_transformed.head(20)

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score,hired
0,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10,No
1,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9,Yes
2,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1,No
3,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7,Yes
4,Alec,Abbott,juanita_hansen@gmail.com,2019-08-17,Zimbabwe,8,Junior,Adobe Experience Manager,2,9,No
5,Allison,Jacobs,alba_rolfson27@yahoo.com,2018-05-18,Wallis and Futuna,19,Trainee,Sales,2,9,No
6,Nya,Skiles,madisen.zulauf@gmail.com,2021-12-09,Myanmar,1,Lead,Mulesoft,2,5,No
7,Mose,Lakin,dale_murazik@hotmail.com,2018-03-13,Italy,18,Lead,Social Media Community Management,7,10,Yes
8,Terrance,Zieme,dustin31@hotmail.com,2022-04-08,Timor-Leste,25,Lead,DevOps,2,0,No
9,Aiyana,Goodwin,vallie.damore@yahoo.com,2019-09-22,Armenia,24,Intern,Development - CMS Backend,4,9,No


# 5. Load the transformed data into a new 'transformed_candidates' table:

##### 5.1 Create the new 'transformed_candidates' table:

In [8]:
with db_engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS transformed_candidates (
            first_name CHARACTER VARYING,
            last_name CHARACTER VARYING,
            email CHARACTER VARYING,
            application_date DATE,
            country CHARACTER VARYING,
            yoe INTEGER,
            seniority CHARACTER VARYING,
            technology CHARACTER VARYING,
            code_challenge_score INTEGER,
            technical_interview_score INTEGER,
            hired BOOLEAN
        );
    """))
    conn.commit()
    print("Table 'transformed_candidates' created successfully.")

# Store DataFrame into PostgreSQL table named 'candidates'
df_candidates_transformed.to_sql("transformed_candidates", db_engine, if_exists="append", index=False)

Table 'transformed_candidates' created successfully.


999

#### 5.2 Read the information from the new 'transformed_candidates' table:

In [9]:
# Just print the first 20 rows to test
df_transformed_verification = pd.read_sql("SELECT * FROM transformed_candidates LIMIT 20;", db_engine)
df_transformed_verification

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score,hired
0,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10,False
1,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9,True
2,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1,False
3,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7,True
4,Alec,Abbott,juanita_hansen@gmail.com,2019-08-17,Zimbabwe,8,Junior,Adobe Experience Manager,2,9,False
5,Allison,Jacobs,alba_rolfson27@yahoo.com,2018-05-18,Wallis and Futuna,19,Trainee,Sales,2,9,False
6,Nya,Skiles,madisen.zulauf@gmail.com,2021-12-09,Myanmar,1,Lead,Mulesoft,2,5,False
7,Mose,Lakin,dale_murazik@hotmail.com,2018-03-13,Italy,18,Lead,Social Media Community Management,7,10,True
8,Terrance,Zieme,dustin31@hotmail.com,2022-04-08,Timor-Leste,25,Lead,DevOps,2,0,False
9,Aiyana,Goodwin,vallie.damore@yahoo.com,2019-09-22,Armenia,24,Intern,Development - CMS Backend,4,9,False
